In [1]:
import pandas as pd

# Load datasets

In [2]:
%%time
discharge_df = pd.read_csv('discharge.csv')

print(f'Shape of the df is {discharge_df.shape}')
print(f'Columns are {discharge_df.columns}')

Shape of the df is (331793, 8)
Columns are Index(['note_id', 'subject_id', 'hadm_id', 'note_type', 'note_seq',
       'charttime', 'storetime', 'text'],
      dtype='object')
CPU times: user 29.4 s, sys: 1.29 s, total: 30.7 s
Wall time: 30.9 s


In [3]:
%%time

diagnoses_icd_df = pd.read_csv('diagnoses_icd.csv')

CPU times: user 910 ms, sys: 149 ms, total: 1.06 s
Wall time: 1.13 s


In [4]:
%%time
icd_codes_df = pd.read_csv('d_icd_diagnoses.csv')

cancer_icd_df = icd_codes_df.loc[icd_codes_df.long_title.str.contains('neoplasm', case=False), :]

cancer_icd_codes_set = set(cancer_icd_df['icd_code'])

CPU times: user 187 ms, sys: 15.5 ms, total: 202 ms
Wall time: 206 ms


# Merge both dataframes

We merge the datasets on subject_id and hadm_id

In [5]:
%%time

merged_df = pd.merge(discharge_df, diagnoses_icd_df, how='inner', on=['subject_id', 'hadm_id'])

CPU times: user 777 ms, sys: 530 ms, total: 1.31 s
Wall time: 1.37 s


In [6]:
# discharge_df[discharge_df['subject_id']==10001186]
# diagnoses_icd_df[diagnoses_icd_df['subject_id']==10001186]
# merged_df[merged_df['subject_id']==10001186]

In [7]:
%%time
cancer_notes_df = merged_df[merged_df['icd_code'].isin(cancer_icd_codes_set)]

CPU times: user 162 ms, sys: 14 ms, total: 176 ms
Wall time: 176 ms


In [8]:
print(f"We have {len(set(cancer_notes_df[cancer_notes_df['seq_num']==1]['subject_id']))} unique people with cancer seq num to be 1")
print(f"We have {len(set(cancer_notes_df[cancer_notes_df['seq_num']!=1]['subject_id']))} unique people with cancer seq num not to be 1")
print(f"""The unique people who do not have seq num as one are {
    len(set(cancer_notes_df[cancer_notes_df['seq_num']!=1]['subject_id']) - set(cancer_notes_df[cancer_notes_df['seq_num']==1]['subject_id']))}""")

We have 14231 unique people with cancer seq num to be 1
We have 35581 unique people with cancer seq num not to be 1
The unique people who do not have seq num as one are 24625


In [9]:
%%time
not_cancer_seq_1_df = merged_df[merged_df['subject_id'].isin(set(merged_df['subject_id']) - set(cancer_notes_df[cancer_notes_df['seq_num']==1]['subject_id']))]

CPU times: user 509 ms, sys: 35.2 ms, total: 544 ms
Wall time: 545 ms


In [10]:
not_cancer_seq_1_df.shape

(3575125, 11)

In [11]:
%%time
cancer_seq_1_df = cancer_notes_df[cancer_notes_df['seq_num']==1]

CPU times: user 3.19 ms, sys: 1.08 ms, total: 4.27 ms
Wall time: 3.43 ms


In [12]:
cancer_seq_1_df.shape

(19448, 11)

In [13]:
# cancer_notes_seq_not_1_df['subject_id'].nunique()
# cancer_notes_df[cancer_notes_df['seq_num']==1]['subject_id'].nunique()

In [14]:
import re

def cleanup(document):
    lines = document.split('\n')
    result = {}
    current_header = None
    current_value = []

    for line in lines:
        # Check if the line ends with a colon, indicating a header
        if line.rstrip().endswith(':'):
            # Save the previous header and its value
            if current_header is not None:
                result[current_header.lower()] = '\n'.join(current_value).strip()
            
            # Extract the header (remove the colon at the end)
            current_header = line.rstrip()[:-1].strip()
            
            # Initialize the current value for the new header
            current_value = []
        else:
            # Append the line to the current value
            current_value.append(line)
    
    # Save the last header and its value
    if current_header is not None:
        result[current_header.lower()] = '\n'.join(current_value).strip()

    return result

def cleanup_v2(document):
    # Use regular expressions to find headers and values
    pattern = re.compile(r'([^:\n]+):\s*([^\n]*)')
    matches = pattern.findall(document)

    result = {}
    current_header = None
    current_value = []

    for header, value in matches:
        # Check if the header is not an empty string
        if header.strip():
            # Save the previous header and its value
            if current_header is not None:
                result[current_header] = '\n'.join(current_value).strip()
            
            # Set the new header and initialize the current value
            current_header = header.strip()
            current_value = [value.strip()]
        else:
            # Append the value to the current value
            current_value.append(value.strip())
    
    # Save the last header and its value
    if current_header is not None:
        result[current_header] = '\n'.join(current_value).strip()

    return result

In [15]:
def cleanup_v3(document):
    lines = document.split('\n')
    result = {}
    current_header = None
    current_value = []

    for line in lines:
        # Check if the line contains a colon, indicating a header
        if ':' in line:
            # Save the previous header and its value
            if current_header is not None:
                result[current_header.lower()] = '\n'.join(current_value).strip()

            # Split the line into header and value
            header, value = map(str.strip, line.split(':', 1))

            # Handle cases where value continues on the next line
            if not value:
                current_header = header
                current_value = []
            else:
                result[header.lower()] = value
                current_header = None
                current_value = []
        elif current_header is not None:
            # Append the line to the current value
            current_value.append(line)

    # Save the last header and its value
    if current_header is not None:
        result[current_header.lower()] = '\n'.join(current_value).strip()

    return result


In [16]:
result = cleanup_v3(not_cancer_seq_1_df['text'][0])
print(result['brief hospital course'])

___ HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, 
bioplar, PTSD, presented from OSH ED with worsening abd 
distension over past week and confusion.  

# Ascites - p/w worsening abd distension and discomfort for last 
week. likely ___ portal HTN given underlying liver disease, 
though no ascitic fluid available on night of admission. No 
signs of heart failure noted on exam. This was ___ to med 
non-compliance and lack of diet restriction. SBP negative


In [17]:
import re

def cleanup_v4(document):
    lines = document.split('\n')
    result = {}
    current_header = None
    current_value = []

    for line in lines:
        # Check if the line ends with a colon, indicating a header
        if re.match(r'^\s*[a-zA-Z ]+\s*:\s*$', line):
            # Save the previous header and its value
            if current_header is not None:
                result[current_header.lower()] = '\n'.join(current_value).strip()

            # Extract the header (remove the colon at the end)
            current_header = line.strip()[:-1]
            
            # Initialize the current value for the new header
            current_value = []
        else:
            # Append the line to the current value
            current_value.append(line)

    # Save the last header and its value
    if current_header is not None:
        result[current_header.lower()] = '\n'.join(current_value).strip()

    return result

In [18]:
cleanup_v4(not_cancer_seq_1_df['text'][100]).keys()

dict_keys(['allergies', 'chief complaint', 'major surgical or invasive procedure', 'history of present illness', 'past medical history', 'social history', 'family history', 'physical exam', 'pertinent results', 'impression', 'brief hospital course', 'medications on admission', 'discharge medications', 'discharge disposition', 'discharge diagnosis', 'primary diagnoses', 'discharge condition', 'discharge instructions', 'we made the following changes to your medicines', 'followup instructions'])

In [19]:
reqd_headers = ['allergies', 'service', 'chief complaint', 'past medical history', \
                'family history', 'primary diagnoses', 'brief hospital course']

In [24]:
def get_reqd_text(text):
    result = cleanup_v4(text)
    out = ''
    for header in reqd_headers:
        if header in result:
            out += f'{header}: {result[header]}\n'
    return out

In [25]:
not_cancer_seq_1_df.columns

Index(['note_id', 'subject_id', 'hadm_id', 'note_type', 'note_seq',
       'charttime', 'storetime', 'text', 'seq_num', 'icd_code', 'icd_version'],
      dtype='object')

In [35]:
%%time

class_0_df = not_cancer_seq_1_df.copy()

class_0_df['class'] = 0

class_0_df.drop(['note_id', 'hadm_id', 'note_type', 'note_seq', \
       'charttime', 'storetime', 'seq_num'], axis=1, inplace=True)

CPU times: user 768 ms, sys: 463 ms, total: 1.23 s
Wall time: 1.29 s


In [36]:
%%time

class_1_df = cancer_seq_1_df.copy()

class_1_df['class'] = 1

class_1_df.drop(['note_id', 'hadm_id', 'note_type', 'note_seq', \
       'charttime', 'storetime', 'seq_num'], axis=1, inplace=True)

CPU times: user 14.5 ms, sys: 234 ms, total: 248 ms
Wall time: 1.02 s


In [37]:
class_1_df.head()

,subject_id,text,icd_code,icd_version,class
73,10000560,\nName: ___ Unit No: _...,1890,9,1
154,10000935,\nName: ___ Unit No: ___...,1977,9,1
181,10000935,\nName: ___ Unit No: ___...,1970,9,1
419,10001401,\nName: ___ Unit No: ___\n \n...,C675,10,1
567,10001877,\nName: ___ ___ No: ___\n \...,2252,9,1


In [39]:
class_1_df.shape

(19448, 5)

In [38]:
class_0_df.head()

,subject_id,text,icd_code,icd_version,class
0,10000032,\nName: ___ Unit No: _...,5723,9,0
1,10000032,\nName: ___ Unit No: _...,78959,9,0
2,10000032,\nName: ___ Unit No: _...,5715,9,0
3,10000032,\nName: ___ Unit No: _...,07070,9,0
4,10000032,\nName: ___ Unit No: _...,496,9,0


In [40]:
class_0_df.shape

(3575125, 5)

In [41]:
%%time
class_1_df['shortened_text'] = class_1_df['text'].apply(get_reqd_text)

CPU times: user 3.87 s, sys: 49.1 ms, total: 3.92 s
Wall time: 4.03 s


In [42]:
class_1_df['shortened_text'][73]

'allergies: Patient recorded as having No Known Allergies to Drugs\n \nAttending: ___.\nchief complaint: renal mass\npast medical history: PMH: nonspecific right axis deviation  \n\nPSH- cesarean section\n\nALL-NKDA\nfamily history: no history of RCC\nbrief hospital course: Patient was admitted to Urology after undergoing laparoscopic \nright radical nephrectomy. No concerning intraoperative events \noccurred; please see dictated operative note for details. The \npatient received perioperative antibiotic prophylaxis. The \npatient was transferred to the floor from the PACU in stable \ncondition.  On POD0, pain was well controlled on PCA, hydrated \nfor urine output >30cc/hour, provided with pneumoboots and \nincentive spirometry for prophylaxis, and ambulated once. On \nPOD1,foley was removed without difficulty, basic metabolic panel \nand complete blood count were checked, pain control was \ntransitioned from PCA to oral analgesics, diet was advanced to a \nclears/toast and crackers d

In [43]:
%%time
class_0_df['shortened_text'] = class_0_df['text'].apply(get_reqd_text)

CPU times: user 14min 26s, sys: 16.2 s, total: 14min 43s
Wall time: 15min 23s


In [45]:
class_0_df['subject_id'].nunique()

131613

In [46]:
class_1_df['subject_id'].nunique()

14231

In [47]:
final_class_0_df = class_0_df.copy()
final_class_1_df = class_1_df.copy()

In [48]:
final_class_0_df = final_class_0_df.drop_duplicates(subset=['subject_id'])
final_class_1_df = final_class_1_df.drop_duplicates(subset=['subject_id'])

In [49]:
final_class_0_df.head()

,subject_id,text,icd_code,icd_version,class,shortened_text
0,10000032,\nName: ___ Unit No: _...,5723,9,0,allergies: No Known Allergies / Adverse Drug R...
39,10000084,\nName: ___ Unit No: __...,G3183,10,0,allergies: No Known Allergies / Adverse Drug R...
45,10000117,\nName: ___ Unit No: ___\n...,R1310,10,0,allergies: omeprazole\n \nAttending: ___.\nchi...
67,10000248,\nName: ___ Unit No: ...,9222,9,0,allergies: No Known Allergies / Adverse Drug R...
76,10000764,\nName: ___ Unit No: ___\n \...,8020,9,0,allergies: No Known Allergies / Adverse Drug R...


In [50]:
final_class_0_df.shape

(131613, 6)

In [53]:
final_class_1_df.head()

,subject_id,text,icd_code,icd_version,class,shortened_text
73,10000560,\nName: ___ Unit No: _...,1890,9,1,allergies: Patient recorded as having No Known...
154,10000935,\nName: ___ Unit No: ___...,1977,9,1,allergies: Sulfa (Sulfonamide Antibiotics) / C...
419,10001401,\nName: ___ Unit No: ___\n \n...,C675,10,1,allergies: No Known Allergies / Adverse Drug R...
567,10001877,\nName: ___ ___ No: ___\n \...,2252,9,1,allergies: Patient recorded as having No Known...
808,10001919,\nName: ___ Unit No: ___\...,C169,10,1,allergies: No Known Allergies / Adverse Drug R...


In [54]:
final_class_1_df.shape

(14231, 6)

In [57]:
final_class_1_df = final_class_1_df.reset_index()
final_class_0_df = final_class_0_df.reset_index()

In [58]:
final_class_1_df.head()

,level_0,index,subject_id,text,icd_code,icd_version,class,shortened_text
0,0,73,10000560,\nName: ___ Unit No: _...,1890,9,1,allergies: Patient recorded as having No Known...
1,1,154,10000935,\nName: ___ Unit No: ___...,1977,9,1,allergies: Sulfa (Sulfonamide Antibiotics) / C...
2,2,419,10001401,\nName: ___ Unit No: ___\n \n...,C675,10,1,allergies: No Known Allergies / Adverse Drug R...
3,3,567,10001877,\nName: ___ ___ No: ___\n \...,2252,9,1,allergies: Patient recorded as having No Known...
4,4,808,10001919,\nName: ___ Unit No: ___\...,C169,10,1,allergies: No Known Allergies / Adverse Drug R...


In [ ]:
# DO NOT RUN RIGHT NOW

final_class_1_df = pd.concat([final_class_1_df, final_class_1_df])
final_class_0_df = final_class_0_df.head(final_class_1_df.shape[0]) 

In [71]:
final_class_1_df = final_class_1_df.drop(['index', 'subject_id', 'text', 'icd_code', 'icd_version'], axis=1)

In [72]:
final_class_0_df = final_class_0_df.drop(['index', 'subject_id', 'text', 'icd_code', 'icd_version'], axis=1)

In [75]:
final_class_1_df

,level_0,class,shortened_text
0,0,1,allergies: Patient recorded as having No Known...
1,1,1,allergies: Sulfa (Sulfonamide Antibiotics) / C...
2,2,1,allergies: No Known Allergies / Adverse Drug R...
3,3,1,allergies: Patient recorded as having No Known...
4,4,1,allergies: No Known Allergies / Adverse Drug R...
...,...,...,...
14226,14226,1,allergies: No Known Allergies / Adverse Drug R...
14227,14227,1,allergies: No Known Allergies / Adverse Drug R...
14228,14228,1,allergies: Ampicillin / Wellbutrin / Penicilli...
14229,14229,1,allergies: No Known Allergies / Adverse Drug R...


In [76]:
final_class_0_df

,class,shortened_text
0,0,allergies: No Known Allergies / Adverse Drug R...
1,0,allergies: No Known Allergies / Adverse Drug R...
2,0,allergies: omeprazole\n \nAttending: ___.\nchi...
3,0,allergies: No Known Allergies / Adverse Drug R...
4,0,allergies: No Known Allergies / Adverse Drug R...
...,...,...
28457,0,allergies: Iodine / Bee Pollens\n \nAttending:...
28458,0,allergies: No Known Allergies / Adverse Drug R...
28459,0,allergies: No Known Allergies / Adverse Drug R...
28460,0,allergies: No Known Allergies / Adverse Drug R...


In [73]:
%%time
final_class_0_df.to_csv('ekdnam_class_0.csv', index=False)

CPU times: user 828 ms, sys: 60.9 ms, total: 889 ms
Wall time: 923 ms


In [74]:
%%time
final_class_1_df.to_csv('ekdnam_class_1.csv', index=False)

CPU times: user 864 ms, sys: 44.4 ms, total: 908 ms
Wall time: 935 ms


In [77]:
final_class_0_df.head()

,class,shortened_text
0,0,allergies: No Known Allergies / Adverse Drug R...
1,0,allergies: No Known Allergies / Adverse Drug R...
2,0,allergies: omeprazole\n \nAttending: ___.\nchi...
3,0,allergies: No Known Allergies / Adverse Drug R...
4,0,allergies: No Known Allergies / Adverse Drug R...


In [81]:
final_class_1_df = final_class_1_df.drop(['level_0'], axis=1)

In [82]:
final_class_1_df.head()

,class,shortened_text
0,1,allergies: Patient recorded as having No Known...
1,1,allergies: Sulfa (Sulfonamide Antibiotics) / C...
2,1,allergies: No Known Allergies / Adverse Drug R...
3,1,allergies: Patient recorded as having No Known...
4,1,allergies: No Known Allergies / Adverse Drug R...


In [96]:
dataset_df = pd.concat([final_class_1_df, final_class_0_df])
dataset_df = dataset_df.reset_index()

In [97]:
dataset_df.shape

(56924, 3)

In [98]:
from sklearn.model_selection import StratifiedKFold

In [99]:
stratified_kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)

In [100]:
X = dataset_df['shortened_text']
y = dataset_df['class']

train_index, test_index = stratified_kfold.split(X, y)

In [101]:
train_index

(array([    1,     2,     3, ..., 56915, 56918, 56921]),
 array([    0,     4,     7, ..., 56920, 56922, 56923]))

In [102]:
test_index

(array([    0,     4,     7, ..., 56920, 56922, 56923]),
 array([    1,     2,     3, ..., 56915, 56918, 56921]))

In [104]:
X_train, X_test = X[train_index[0]], X[test_index[0]]
y_train, y_test = y[train_index[0]], y[test_index[0]]

In [105]:
X_train

1        allergies: Sulfa (Sulfonamide Antibiotics) / C...
2        allergies: No Known Allergies / Adverse Drug R...
3        allergies: Patient recorded as having No Known...
5        allergies: Codeine\n \nAttending: ___.\nchief ...
6        allergies: No Known Allergies / Adverse Drug R...
                               ...                        
56909    allergies: No Known Allergies / Adverse Drug R...
56911    allergies: No Known Allergies / Adverse Drug R...
56915    allergies: No Known Allergies / Adverse Drug R...
56918    allergies: Gabapentin\n \nAttending: ___\nchie...
56921    allergies: No Known Allergies / Adverse Drug R...
Name: shortened_text, Length: 28462, dtype: object

In [106]:
X_test

0        allergies: Patient recorded as having No Known...
4        allergies: No Known Allergies / Adverse Drug R...
7        allergies: Percocet / cucumber\n \nAttending: ...
9        allergies: Penicillins / Shellfish Derived\n \...
11       allergies: No Known Allergies / Adverse Drug R...
                               ...                        
56917    allergies: Patient recorded as having No Known...
56919    allergies: Iodine / Bee Pollens\n \nAttending:...
56920    allergies: No Known Allergies / Adverse Drug R...
56922    allergies: No Known Allergies / Adverse Drug R...
56923    allergies: venom-honey bee\n \nAttending: ___....
Name: shortened_text, Length: 28462, dtype: object

In [123]:
train_df = pd.DataFrame()
train_df['X'] = X_train
train_df['y'] = y_train
print(train_df.isnull().sum())
train_df = train_df.dropna()

train_df.to_csv('ekdnam_train.csv', index=False)

X    0
y    0
dtype: int64


In [124]:
train_df.shape

(28462, 2)

In [125]:
test_df = pd.DataFrame()
test_df['X'] = X_test
test_df['y'] = y_test
print(test_df.isnull().sum())
test_df = test_df.dropna()

test_df.to_csv('ekdnam_test.csv', index=False)

X    0
y    0
dtype: int64


In [117]:
train_df.isnull().sum()

X    0
y    0
dtype: int64

In [118]:
test_df.isnull().sum()

X    0
y    0
dtype: int64